In [65]:
import torch

model_path = "debug/20230605150559/spectral_fc_01.pth"
dict = torch.load(model_path, map_location=torch.device('cpu'))
print("epoch",dict['epoch'])
for k, v in dict["state_dict"].items():
    if "lambda" in k:
        print(k,v)


epoch 60
module.lambda_l1 tensor([ 0.5968,  0.4106,  0.9196, -0.0564,  0.1101, -0.1365,  0.0909, -0.8715,
         0.4072, -0.3280])
module.lambda_tv tensor([ 0.0010,  0.5761,  0.7933,  0.5102, -0.6357,  0.2458,  0.1870,  0.5339,
         0.3418,  0.6492])
module.lambda_dwt tensor([0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010])
